<a href="https://colab.research.google.com/github/parkrye/Python/blob/main/202210_Bigdata/06_Key_Value_RDD_Operations%2C_Joins.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster("local").setAppName("key-value_rdd_op_joins")
sc = SparkContext(conf = conf)

# Operations

- `groupByKey`
    - `KeyValueRDD.groupByKey()`
    - 그룹핑 후에 특정 Transformations 같은 연산
    - `Key` 값이 있는 상태에서 시작
- `groupBy()`
    - `RDD.groupBy(numPartitions=None, partitionFunc=<function portable_hash>)`
    - 함수에 의해서 그룹이 생기는 연산

In [ ]:
rdd = sc.parallelize([
    ("짜장면", 15),
    ("짬뽕", 10),
    ("짜장면", 5)
])

g_rdd = rdd.groupByKey()
g_rdd.collect()

[('짜장면', <pyspark.resultiterable.ResultIterable at 0x2638291d9a0>),
 ('짬뽕', <pyspark.resultiterable.ResultIterable at 0x2638291da30>)]

In [ ]:
g_rdd.mapValues(len).collect()

[('짜장면', 2), ('짬뽕', 1)]

In [ ]:
g_rdd.mapValues(list).collect()

[('짜장면', [15, 5]), ('짬뽕', [10])]

In [ ]:
# groupBy는 그룹핑할 키에 대한 정의를 개발자가 직접 해줘야 한다.
grouped = sc.parallelize([
    "C", "C#", "C++", 'Python', "Java", "JavaScript"
]).groupBy(lambda x : x[0]).collect()

grouped

[('C', <pyspark.resultiterable.ResultIterable at 0x263828f3dc0>),
 ('P', <pyspark.resultiterable.ResultIterable at 0x2638297d250>),
 ('J', <pyspark.resultiterable.ResultIterable at 0x2638297d3a0>)]

In [ ]:
for k, v in grouped:
    print(k, list(v))

C ['C', 'C#', 'C++']
P ['Python']
J ['Java', 'JavaScript']


In [ ]:
# groupByKey는 K-V RDD를 사용할 때 Key가 알아서 그룹핑의 기준이 된다.

x = sc.parallelize([
    ("MATH", 7), ("MATH", 2), ("ENGLISH", 7),
    ("SCIENCE", 7), ("ENGLISH", 4), ("ENGLISH", 9),
    ("MATH", 8), ("MATH", 3), ("ENGLISH", 4),
    ("SCIENCE", 6), ("SCIENCE", 9), ("SCIENCE", 5)
], 3)

y = x.groupByKey()

In [ ]:
print(y.getNumPartitions())

3


In [ ]:
# 파티션의 개수를 바꿀 수 있다.
y = x.groupByKey(2)
y.getNumPartitions()

2

In [ ]:
y.collect()

[('MATH', <pyspark.resultiterable.ResultIterable at 0x26382988a00>),
 ('ENGLISH', <pyspark.resultiterable.ResultIterable at 0x2638298a340>),
 ('SCIENCE', <pyspark.resultiterable.ResultIterable at 0x2638298a490>)]

In [ ]:
for t in y.collect():
    print(t[0], list(t[1])) # t[0] : Key, t[1] : 그룹핑에 의해 묶인 값

MATH [7, 2, 8, 3]
ENGLISH [7, 4, 9, 4]
SCIENCE [7, 6, 9, 5]


# reduceByKey
- `KeyValueRDD.reduceByKey(<func>, numPartitions=None, partitionFunc=<function portable_hash>)`
- 주어지는 `key`를 기준으로 `Group`을 만들고 합쳐줍니다.
- Transformations 함수 입니다.

In [ ]:
from operator import add

rdd = sc.parallelize([
    ("짜장면", 15),
    ("짬뽕", 10),
    ("짜장면", 5)
])

rdd.reduceByKey(add).collect()

[('짜장면', 20), ('짬뽕', 10)]

개념적으로는 `groupByKey + reduce`입니다. `groupByKey`보다 `reduceBykey`를 쓰는게 훨씬 빠릅니다.

In [ ]:
x = sc.parallelize([
    ("MATH", 7), ("MATH", 2), ("ENGLISH", 7),
    ("SCIENCE", 7), ("ENGLISH", 4), ("ENGLISH", 9),
    ("MATH", 8), ("MATH", 3), ("ENGLISH", 4),
    ("SCIENCE", 6), ("SCIENCE", 9), ("SCIENCE", 5)
], 3)

x.reduceByKey(lambda a, b : a + b).collect()

[('MATH', 20), ('ENGLISH', 24), ('SCIENCE', 27)]

# mapValues
- `KeyValueRDD.mapValues(<func>)`
- 함수를 `Value`에만 적용합니다.
    - 파티션과 키는 그 위치에 그대로 있습니다.
- `Transformations`

In [ ]:
rdd = sc.parallelize([
    ("하의", ["청바지", "반바지", "치마"]),
    ("상의", ["니트", "반팔", "긴팔", "나시"])
])

rdd.mapValues(lambda x : len(x)).collect()

[('하의', 3), ('상의', 4)]

`key`가 아닌 `value`에만 적용할 함수를 만들 수가 있기 때문에 데이터의 파티션이 변경될 염려가 없다.

# countByKey
- `KeyValueRDD.countByKey(<func>)`
- 각 키가 가진 요소들의 개수를 센다.
- `Action`

In [ ]:
rdd = sc.parallelize([
    ("하의", ["청바지", "반바지", "치마"]),
    ("상의", ["니트", "반팔", "긴팔", "나시"])
])

rdd.countByKey()

defaultdict(int, {'하의': 1, '상의': 1})

# keys()
- 모든 key를 가진 RDD를 생성합니다.
- `keys()`는 파티션을 유지하거나 키가 굉장히 많은 경우가 있기 때문에 Transformations 입니다.

In [ ]:
rdd.keys()

PythonRDD[43] at RDD at PythonRDD.scala:53

In [ ]:
rdd.keys().collect()

['하의', '상의']

In [ ]:
x = sc.parallelize([
    ("MATH", 7), ("MATH", 2), ("ENGLISH", 7),
    ("SCIENCE", 7), ("ENGLISH", 4), ("ENGLISH", 9),
    ("MATH", 8), ("MATH", 3), ("ENGLISH", 4),
    ("SCIENCE", 6), ("SCIENCE", 9), ("SCIENCE", 5)
], 3)

print(x.keys().count()) # 키의 총 개수
print(x.keys().distinct().count()) # 키의 종류 개수

12
3


# Joins

In [ ]:
# Inner Join : 서로간에 존재하는 키만 합쳐줍니다.
rdd1 = sc.parallelize([
    ("foo", 1),
    ("goo", 2),
    ("hoo", 3)
])

rdd2 = sc.parallelize([
    ("foo", 1),
    ("goo", 2),
    ("goo", 10),
    ("moo", 6)
])

rdd1.join(rdd2).collect()

[('foo', (1, 1)), ('goo', (2, 2)), ('goo', (2, 10))]

**Outer Join**
- 기준이 되는 한 쪽에는 데이터가 있고, 다른 쪽에는 데이터가 없는 경우
    - 설정한 기준에 따라서 기준에 맞는 데이터가 항상 남아있는다.
- `leftOuterJoin` : 왼쪽에 있는 rdd가 기준이 됩니다. (함수를 호출하는 쪽)
- `rightOuterJoin` : 오른쪽에 있는 rdd가 기준이 됩니다. (함수에 매개변수로 들어가는 쪽)

In [ ]:
rdd1.leftOuterJoin(rdd2).collect()

[('foo', (1, 1)), ('goo', (2, 2)), ('goo', (2, 10)), ('hoo', (3, None))]

In [ ]:
rdd1.rightOuterJoin(rdd2).collect()

[('foo', (1, 1)), ('moo', (None, 6)), ('goo', (2, 2)), ('goo', (2, 10))]

In [ ]:
sc.stop()